### define environment

In [2]:
USING_GOOGLE_COLAB = False

In [3]:
if USING_GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [4]:
if USING_GOOGLE_COLAB:
    !git clone https://github.com/GrzegorzKazana/nearest-neighbours.git
    !ls drive

In [5]:
import os
import sys

proj_base_path = '/content/nearest-neighbours' if USING_GOOGLE_COLAB else './'
data_base_path = '/content/drive/My Drive/' if USING_GOOGLE_COLAB else './data'
models_base_path = '/content/drive/My Drive/' if USING_GOOGLE_COLAB else './models'
sys.path.append(os.path.join(os.getcwd(), proj_base_path))

### code starts here

In [16]:
import numpy as np

images_file_path = 'imgs_task1_smol.npy'
labels_file_path = 'labels_task1_smol.npy'

images = np.load(os.path.join(data_base_path, images_file_path))
labels = np.load(os.path.join(data_base_path, labels_file_path))
labels = np.squeeze(labels)

print(images.shape, labels.shape)
print(f'number of samples {len(images)}')

assert len(images) == len(labels)

n_of_samples = len(images)

(362, 128, 128, 3) (362, 48)
number of samples 362


In [17]:
def data_generator(X, Y, batch_size):
    while True:
        batch_idxs = np.random.randint(0, len(X), size=batch_size)

        x = X[batch_idxs]
        y = Y[batch_idxs]
        
        yield x, y
        
data_gen = data_generator(images, labels, 16)
next(data_gen)

(array([[[[ 28,  46,  53],
          [ 21,  37,  48],
          [ 31,  46,  56],
          ...,
          [137, 144, 138],
          [133, 140, 134],
          [129, 136, 131]],
 
         [[ 28,  43,  51],
          [ 25,  39,  51],
          [ 29,  42,  54],
          ...,
          [132, 139, 134],
          [131, 138, 133],
          [128, 135, 130]],
 
         [[ 31,  45,  57],
          [ 23,  36,  50],
          [ 33,  46,  60],
          ...,
          [128, 137, 134],
          [128, 137, 134],
          [127, 134, 131]],
 
         ...,
 
         [[ 13,  20,  27],
          [ 16,  22,  29],
          [ 16,  22,  29],
          ...,
          [158, 208, 228],
          [160, 210, 230],
          [160, 211, 231]],
 
         [[ 17,  22,  31],
          [ 16,  22,  29],
          [ 13,  18,  25],
          ...,
          [151, 204, 225],
          [159, 212, 233],
          [158, 208, 228]],
 
         [[ 19,  25,  30],
          [ 29,  35,  42],
          [ 16,  22,  29],
   

### model preparation

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

INPUT_SHAPE = (128, 128, 3)
HIDDEN_SIZE = 256
OUTPUT_SIZE = labels.shape[-1]

In [12]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

input_layer = Input(shape=INPUT_SHAPE)
model_vgg16_conv = DenseNet121(weights='imagenet', include_top=False)

for l in model_vgg16_conv.layers:
    l.trainable = False

output_vgg16_conv = model_vgg16_conv(input_layer)

x = Flatten()(output_vgg16_conv)
x = Dense(HIDDEN_SIZE, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(OUTPUT_SIZE, activation='sigmoid')(x)

model = Model(input_layer, outputs=x)

model.summary()

29089792/29084464 [==============================] - 3s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
densenet121 (Model)          multiple                  7037504   
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               4194560   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 48)                12336     
Total params: 11,244,400
Trainable params: 4,206,896
Non-train

In [13]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy', 'binary_crossentropy', 'categorical_crossentropy']
)

### training

In [18]:
from datetime import datetime

def preprocessed_gen(X, Y, batch_size):
    gen = data_generator(X, Y, batch_size)
    while True:
        x, y = next(gen)
        yield preprocess_input(x), y

BATCH_SIZE = 32

indices = np.random.permutation(n_of_samples)
train_split_idx = int(0.8 * n_of_samples)
training_idx, test_idx = indices[:train_split_idx], indices[train_split_idx:]
training_img, test_img = images[training_idx,:], images[test_idx,:]
training_labels, test_labels = labels[training_idx,:], labels[test_idx,:]
# training_img, test_img = images, images
# training_labels, test_labels = labels, labels

train_data_gen = preprocessed_gen(training_img, training_labels, BATCH_SIZE)
test_data_gen = preprocessed_gen(test_img, test_labels, BATCH_SIZE)

In [ ]:
STEPS_PER_EPOCH = 20
VALIDATION_STEPS = 2
EPOCHS = 10

# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=16)

t = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
model_path = os.path.join(models_base_path, f'm_{t}.h5')

mc = ModelCheckpoint(model_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)


history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
#     callbacks=[es, mc],
#     callbacks=[es],
    validation_data=test_data_gen,
    validation_steps=VALIDATION_STEPS
)
# for _ in range(1000):
#     idx = np.random.randint(len(training_img))
#     x, y = training_img[np.newaxis, idx], training_labels[np.newaxis, idx]
#     num_of_ones = len(np.argwhere(y > 0.5))
#     w = {i: 1 for i in range(OUTPUT_SIZE)}
#     w = {
#         **w,
#         **{a[0]: OUTPUT_SIZE / num_of_ones for a in np.argwhere(y > 0.5)}
#     }
    
#     model.fit(
#         x,
#         y,
#         class_weight=w
#     )

Epoch 1/10
20/20 [==============================] - 85s 4s/step - loss: 0.5772 - binary_accuracy: 0.8837 - binary_crossentropy: 0.5772 - categorical_crossentropy: 26.3238 - val_loss: 0.5449 - val_binary_accuracy: 0.9023 - val_binary_crossentropy: 0.5449 - val_categorical_crossentropy: 26.9532
Epoch 2/10
20/20 [==============================] - 84s 4s/step - loss: 0.1920 - binary_accuracy: 0.9354 - binary_crossentropy: 0.1920 - categorical_crossentropy: 15.6556 - val_loss: 0.3280 - val_binary_accuracy: 0.9176 - val_binary_crossentropy: 0.3280 - val_categorical_crossentropy: 21.6079
Epoch 3/10
20/20 [==============================] - 86s 4s/step - loss: 0.1029 - binary_accuracy: 0.9608 - binary_crossentropy: 0.1029 - categorical_crossentropy: 13.1417 - val_loss: 0.2679 - val_binary_accuracy: 0.9316 - val_binary_crossentropy: 0.2679 - val_categorical_crossentropy: 19.2846
Epoch 4/10
 1/20 [>.............................] - ETA: 1:15 - loss: 0.1125 - binary_accuracy: 0.9551 - binary_crosse

In [121]:
from datetime import datetime

t = datetime.timestamp(datetime.now())
models_file_name = f'm_{t}.h5'

model.save(os.path.join(models_base_path, models_file_name))

In [ ]:
from tensorflow.keras.models import load_model

load_model(os.path.join(models_base_path, 'm_1572059121.544981.h5'))

In [112]:
x, y = next(test_data_gen)
np.argwhere(y > 0)

array([[ 0,  6],
       [ 0, 13],
       [ 0, 21],
       [ 0, 22],
       [ 0, 24],
       [ 0, 30],
       [ 0, 34],
       [ 1,  6],
       [ 1, 16],
       [ 1, 22],
       [ 1, 34],
       [ 1, 39],
       [ 2,  0],
       [ 2, 21],
       [ 2, 22],
       [ 2, 30],
       [ 2, 34],
       [ 2, 37],
       [ 2, 42],
       [ 3, 21],
       [ 3, 22],
       [ 3, 30],
       [ 3, 34],
       [ 3, 46],
       [ 4,  0],
       [ 4, 22],
       [ 4, 30],
       [ 4, 34],
       [ 5, 19],
       [ 5, 30],
       [ 6,  6],
       [ 6, 13],
       [ 6, 22],
       [ 6, 30],
       [ 6, 31],
       [ 6, 34],
       [ 6, 39],
       [ 7,  3],
       [ 7,  4],
       [ 7,  5],
       [ 7, 22],
       [ 7, 27],
       [ 7, 30],
       [ 7, 34],
       [ 7, 46],
       [ 8, 10],
       [ 8, 22],
       [ 8, 30],
       [ 8, 34],
       [ 8, 39],
       [ 9, 12],
       [ 9, 21],
       [ 9, 22],
       [ 9, 30],
       [ 9, 34],
       [ 9, 46],
       [10,  3],
       [10,  4],
       [10, 22

In [113]:
res = model.predict(x)

In [115]:
np.argwhere(res > 0.5)

array([], shape=(0, 2), dtype=int64)